In [1]:
import pandas as pd
import numpy as np

In [20]:
import time

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,LSTM,LeakyReLU

In [3]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [4]:
df=pd.read_csv('Public Data.csv')

In [5]:
df['Status']=df['Status'].replace(['Developing', 'Developed'],[0,1])

In [6]:
df.isna().sum()

Unnamed: 0                    0
Country                       0
Year                          0
Status                        0
Power Level                  10
Civilian Deaths              10
Military Accidents            0
Substance Abuse              17
Military Investment           0
Air Force                   544
Army                          0
Intelligence                 32
Navy                          0
Submarines                   19
Educational Investment       45
Literacy                     19
Revolts                       0
GDP                         419
Population                  611
Offense                      32
Defense                      32
Political Stability         157
International Reputation    153
dtype: int64

In [7]:
df_=df.copy()

In [8]:
df=df.drop(['Unnamed: 0','Country'],axis=1)

In [9]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(df)
df=imputer.transform(df)
df=pd.DataFrame(df)

In [10]:
df_.columns

Index(['Unnamed: 0', 'Country', 'Year', 'Status', 'Power Level',
       'Civilian Deaths', 'Military Accidents', 'Substance Abuse',
       'Military Investment', 'Air Force', 'Army', 'Intelligence', 'Navy',
       'Submarines', 'Educational Investment', 'Literacy', 'Revolts', 'GDP',
       'Population', 'Offense', 'Defense', 'Political Stability',
       'International Reputation'],
      dtype='object')

In [11]:
df.columns=['Year', 'Status', 'Power Level',
       'Civilian Deaths', 'Military Accidents', 'Substance Abuse',
       'Military Investment', 'Air Force', 'Army', 'Intelligence', 'Navy',
       'Submarines', 'Educational Investment', 'Literacy', 'Revolts', 'GDP',
       'Population', 'Offense', 'Defense', 'Political Stability',
       'International Reputation']

In [12]:
df['Country']=df_['Country']

In [13]:
df.head()

,Year,Status,Power Level,Civilian Deaths,Military Accidents,Substance Abuse,Military Investment,Air Force,Army,Intelligence,...,Educational Investment,Literacy,Revolts,GDP,Population,Offense,Defense,Political Stability,International Reputation,Country
0,2022.0,0.0,59.73,271.0,62.0,-0.25,73.83,59.0,495.0,18.33,...,8.06,62.0,-0.21,612.86,327582.0,17.16,17.68,0.44,10.57,Afghanistan
1,2021.0,0.0,60.50,268.0,66.0,-0.67,73.66,71.0,431.0,18.80,...,8.08,65.0,-0.04,631.58,31731688.0,18.28,17.85,0.72,10.40,Afghanistan
2,2020.0,0.0,60.09,271.0,70.0,-0.17,77.76,65.0,2788.0,16.39,...,8.71,70.0,-0.21,669.50,3696953.0,16.85,18.46,0.99,9.52,Afghanistan
3,2019.0,0.0,58.88,279.0,70.0,0.69,6.95,72.0,3014.0,16.64,...,8.34,72.0,-0.60,63.67,2978596.0,17.72,18.11,0.84,9.82,Afghanistan
4,2018.0,0.0,59.26,272.0,74.0,0.07,79.82,70.0,1988.0,17.04,...,9.40,66.0,-0.12,553.89,2883166.0,18.91,18.76,0.52,9.14,Afghanistan


In [14]:
test_df=pd.read_csv('Test Dataset.csv')

In [15]:
test_df

,Country,Year,Status,Civilian Deaths,Military Accidents,Military Investment,Air Force,Army,Intelligence,Navy,Submarines,Literacy,Revolts,GDP,Population,Offense,Defense,Political Stability,International Reputation
0,Afghanistan,2023,Developing,264,61,71.59,69.0,1153,18.28,84,8,68,0.65,583.49,33736494.0,17.50,16.52,1.33,9.68
1,Albania,2023,Developing,75,2,364.95,98.0,-2,57.89,2,97,94,-0.88,3954.09,28873.0,1.43,1.58,1.81,14.35
2,Algeria,2023,Developing,20,21,-0.23,95.0,63,59.52,22,93,92,0.32,4132.55,39871526.0,5.97,5.76,0.25,13.24
3,Angola,2023,Developing,337,65,-0.11,58.0,117,23.71,102,9,63,1.44,3696.01,2785939.0,7.46,8.82,0.04,10.25
4,Antigua and Barbuda,2023,Developing,9,-6,-0.67,98.0,1,47.73,1,95,100,0.68,13567.04,NaN,3.53,3.16,1.18,14.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,Venezuela (Bolivarian Republic of),2023,Developing,156,10,-0.03,87.0,-3,62.08,12,88,88,0.82,NaN,NaN,1.06,1.07,0.04,14.52
179,Viet Nam,2023,Developing,128,28,0.03,98.0,261,17.27,33,99,94,-0.19,NaN,NaN,14.00,14.87,-0.21,13.09
180,Yemen,2023,Developing,221,39,0.32,67.0,469,40.77,46,64,75,-0.26,NaN,NaN,13.46,13.32,0.76,8.74
181,Zambia,2023,Developing,35,24,-0.22,12.0,13,23.61,41,8,15,3.66,1313.08,161586.0,6.17,5.99,-0.30,12.46


In [16]:
contries=test_df['Country'].unique()

In [17]:
cont_1=[]
for cont in contries:
    if (len(df[df['Country']==cont]) <15 ):
        cont_1.append(cont)
        print(cont + f"{len(df[df['Country']==cont])}")

In [43]:
model=tf.keras.Sequential()

model.add(LSTM(128,input_shape=(10,20),return_sequences=True))
model.add(LeakyReLU(alpha=0.5))

model.add(LSTM(64,return_sequences=True))
model.add(LeakyReLU(alpha=0.5))

model.add(LSTM(32,return_sequences=True))
model.add(LeakyReLU(alpha=0.5))

model.add(LSTM(32))
model.add(LeakyReLU(alpha=0.5))

model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))

model.add(Dense(1))

In [44]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
             loss=tf.keras.losses.MeanSquaredError())

In [41]:
%%time
pred=[]
for cont in contries:
    train=df[df['Country']==cont]
    train=train.sort_values('Year')
    train['Year']=pd.to_datetime(train['Year'],format='%Y')
    train=train.drop(['Country'],axis=1)
    if (len(train)==1):
        pred.append(np.array(train['Power Level']))
        print(cont + f" done and predicted {pre}")
    else : 
        X=train.drop('Year',axis=1).to_numpy().tolist()
        y=train['Power Level'].tolist()
        df_gen=TimeseriesGenerator(X,y,length=5,sampling_rate=1,batch_size=1,stride=1)
        model.fit(df_gen,epochs=75,verbose=0)
        pr=train.iloc[5:].drop('Year',axis=1)
        pr=np.array(pr).reshape(1,10,20)
        pre=model.predict(pr)
        pred.append(pre)
        print(cont + f" done and predicted {pre}")

Afghanistan done and predicted [[59.070755]]
Albania done and predicted [[75.87774]]
Algeria done and predicted [[74.504585]]
Angola done and predicted [[49.506485]]
Antigua and Barbuda done and predicted [[75.51641]]
Argentina done and predicted [[75.03321]]
Armenia done and predicted [[74.44507]]
Australia done and predicted [[82.38685]]
Austria done and predicted [[84.22402]]
Azerbaijan done and predicted [[72.71668]]
Bahamas done and predicted [[74.46088]]
Bahrain done and predicted [[75.98629]]
Bangladesh done and predicted [[71.59937]]
Barbados done and predicted [[74.69699]]
Belarus done and predicted [[71.15951]]
Belgium done and predicted [[81.94318]]
Belize done and predicted [[69.892006]]
Benin done and predicted [[57.86754]]
Bhutan done and predicted [[67.54144]]
Bolivia (Plurinational State of) done and predicted [[69.68205]]
Bosnia and Herzegovina done and predicted [[75.88146]]
Botswana done and predicted [[60.77099]]
Brazil done and predicted [[73.975136]]
Brunei Daruss

In [32]:
pred=np.array(pred).reshape(183)

In [33]:
pred=pd.DataFrame(pred)

In [34]:
pred['id']=pred.index

In [36]:
pred['Power Level']=pred[0]

In [37]:
pred=pred.drop(0,axis=1)

In [38]:
pred.to_csv('pred.csv',index=False)